In [ ]:
from image_functions import image_sampler, oklab_to_linear_srgb, linear_srgb_to_oklab
import skimage as ski
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from jax import vmap
import jax.random as jr
from IPython.display import HTML

matplotlib.rcParams["animation.embed_limit"] = 2**128
plt.rcParams["axes.spines.right"] = False
plt.rcParams["axes.spines.top"] = False



In [ ]:
# read in original image
im = mpimg.imread("bluecat.png")

# process image. downscaling is important, as it requires simulation of less particles
# im = im / 255 # uncomment if reading jpeg
im = ski.color.rgba2rgb(im) # uncomment if reading png
im = ski.transform.rescale(im, 0.25, channel_axis=-1, anti_aliasing=True, order=3)

In [ ]:
# define and run sampler
manager = image_sampler(
    im,
    num_particles=15000,
    loss_space="oklab",
    posterization_params={"posterizer": "oklab", "n_colors": 9},
    smoother_params={"kernel_size": 9, "kernel_std": 1},
    likelihood_params={"dist_mod": lambda x: (9 * x) ** 2},
    sampler_params={"lambd_range": (-3, 0.7), "annealing_steps":10},
)
out = manager.run(jr.key(3))


In [ ]:
# view posterized image
plt.imshow(vmap(vmap(oklab_to_linear_srgb))(manager.palette))

In [ ]:
# view reference image for loss function it's important that it's smoothed:
# otherwise, particles are more likely to pool at edges between colors
plt.imshow(vmap(vmap(oklab_to_linear_srgb))(manager.ref_img))

In [ ]:
# render gif
ani = manager.draw_gif(render='img')
HTML(ani.to_jshtml())

In [ ]:
# render gif with smoothing
ani = manager.draw_gif(render='img', smoothing_params={'kernel_size':3, 'kernel_std':0.5})
HTML(ani.to_jshtml())

In [ ]:
# show loss for each particle color. as this plots negative loglikelihood, particles seek
# regions with low (dark) values.
manager.show_loss_map()